<a href="https://colab.research.google.com/github/dakar123/dakar123/blob/main/piedra_papel_tijera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importacion cod 01


In [2]:
import kagglehub, shutil, os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from tensorflow.keras.preprocessing import image
import ipywidgets as widgets
from IPython.display import display, Image
import time
import threading
import random
from google.colab.output import eval_js
from base64 import b64decode
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from IPython.display import Javascript
from PIL import Image
import io
from IPython.display import Image as DisplayImage
from PIL import Image as PILImage
choice_to_spanish = {'rock': 'Piedra ✊', 'paper': 'Papel ✋', 'scissors': 'Tijera ✌️'}


cod 02

In [3]:

# Descargar dataset (queda en caché de KaggleHub)
path = kagglehub.dataset_download("glushko/rock-paper-scissors-dataset")

print("Descargado en:", path)

# Carpeta destino en tu Drive o en /content
dest = "/content/DATOS"

# Crear carpeta si no existe
os.makedirs(dest, exist_ok=True)

# Copiar todo el dataset allí
shutil.copytree(path, dest, dirs_exist_ok=True)


Descargado en: /kaggle/input/rock-paper-scissors-dataset


'/content/DATOS'

cod 03

In [4]:

base_dir = '/content/DATOS'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

cod 04

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

cod 05

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

Found 1020 images belonging to 3 classes.
Found 804 images belonging to 3 classes.
Found 540 images belonging to 3 classes.


cod 06

In [7]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
     Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
     Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


cod 07

In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

cod 08

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=13,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/13
31/31 ━━━━━━━━━━━━━━━━━━━━ 118s 4s/step - accuracy: 0.3593 - loss: 1.0985 - val_accuracy: 0.3338 - val_loss: 1.0960
Epoch 2/13
 1/31 ━━━━━━━━━━━━━━━━━━━━ 1:07 2s/step - accuracy: 0.2188 - loss: 1.1326

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


cod 09

In [ ]:
def determine_winner(player_choice, computer_choice):
    """
    Determina el ganador con manejo de casos especiales.
    """
    # Normalizar las elecciones
    player_choice = player_choice.lower()
    computer_choice = computer_choice.lower()

    # Manejamos el caso donde la predicción falla
    if player_choice == "unknown" or player_choice == "error":
        return "invalid"

    if player_choice == computer_choice:
        return 'tie'
    elif (player_choice == 'rock' and computer_choice == 'scissors') or \
         (player_choice == 'paper' and computer_choice == 'rock') or \
         (player_choice == 'scissors' and computer_choice == 'paper'):
        return 'win'
    else:
        return 'lose'

cod 10

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f"Pérdida en prueba: {loss:.4f}")
print(f"Precisión en prueba: {accuracy:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Graficar accuracy y loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(epochs, acc, 'b-', label='Entrenamiento')
plt.plot(epochs, val_acc, 'r-', label='Validación')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(epochs, loss, 'b-', label='Entrenamiento')
plt.plot(epochs, val_loss, 'r-', label='Validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
def predict_player_choice(img_path, model):
    """
    Predice la elección de piedra, papel o tijera a partir de una imagen utilizando el modelo entrenado.

    Args:
        img_path (str): La ruta al archivo de imagen.
        model (tf.keras.Model): El modelo CNN entrenado.

    Returns:
        str: La elección predicha ('rock', 'paper', or 'scissors').
    """
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale the image

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions)

    # Asumiendo que los índices de clase están ordenados como 'paper', 'rock', 'scissors'
    # Es posible que necesites ajustar esto según cómo tus generadores de datos ordenaron las clases
    class_labels = ['paper', 'rock', 'scissors']

    predicted_choice = class_labels[predicted_class_index]

    return predicted_choice

cod 11

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const video = document.createElement('video');
      div.appendChild(video);
      document.body.appendChild(div);
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      video.srcObject = stream;
      await video.play();
      await new Promise(resolve => setTimeout(resolve, 1000));  // espera 2 seg
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getTracks().forEach(track => track.stop());
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    # Justo antes de return filename
    try:
        img = PILImage.open(filename)
        img.verify()  # Verifica que la imagen sea válida
        print("✅ Imagen válida capturada.")
    except Exception as e:
        print(f"❌ Error al verificar imagen: {e}")

    return filename



cod 12

In [ ]:
# Reemplaza el código actual de la interfaz con este mejorado
output = widgets.Output()

# Crear botones con estilo mejorado
play_button = widgets.Button(
    description="🎮 Jugar",
    button_style='success',
    icon='gamepad'
)
stats_button = widgets.Button(
    description="📊 Estadísticas",
    button_style='info',
    icon='bar-chart'
)
exit_button = widgets.Button(
    description="🚪 Salir",
    button_style='danger',
    icon='sign-out'
)

# Botón para nuevo juego
new_game_button = widgets.Button(
    description="🔄 Nuevo Juego",
    button_style='warning',
    icon='refresh'
)
new_game_button.layout.display = 'none'  # Oculto al inicio

# Crear un indicador de progreso para la cuenta regresiva
countdown_progress = widgets.IntProgress(
    value=5,
    min=0,
    max=5,
    description='Tiempo:',
    bar_style='warning',
    style={'bar_color': 'orange'}
)

# Etiquetas mejoradas
game_info_label = widgets.HTML(value="<h3>¡Bienvenido a Piedra, Papel o Tijera!</h3>")
result_label = widgets.HTML(value="")
stats_label = widgets.HTML(value="")

# Área para mostrar la imagen capturada
image_display = widgets.Output()

player_wins = 0
player_losses = 0
player_ties = 0

def countdown_and_capture(b=None):
    global player_wins, player_losses, player_ties

    game_info_label.value = """
    <h4>¡Prepárate! Coloca tu mano mostrando claramente:</h4>
    <ul>
      <li>✊ Piedra</li>
      <li>✋ Papel</li>
      <li>✌️ Tijera</li>
    </ul>
    """

    for i in range(5, 0, -1):
        countdown_progress.value = i
        time.sleep(1)

    try:
        # Capturar foto
        photo_path = take_photo()

        with image_display:
            image_display.clear_output()
            display(DisplayImage(filename=photo_path))

        if 'model' in globals():
            player_choice = predict_player_choice(photo_path, model)
            computer_choice = random.choice(['rock', 'paper', 'scissors'])

            choice_to_emoji = {'rock': '✊', 'paper': '✋', 'scissors': '✌️'}

            result = determine_winner(player_choice, computer_choice)

            if result == 'win':
                player_wins += 1
                result_color = "green"
            elif result == 'lose':
                player_losses += 1
                result_color = "red"
            else:
                player_ties += 1
                result_color = "blue"

            result_label.value = f"""
            <div style='border: 2px solid {result_color}; padding: 10px; border-radius: 10px;'>
                <h3 style='color: {result_color};'>Resultado: {result.upper()}</h3>
                <p><b>Tú:</b> {choice_to_emoji[player_choice]} {player_choice}</p>
                <p><b>Computadora:</b> {choice_to_emoji[computer_choice]} {computer_choice}</p>
            </div>
            """
        else:
            result_label.value = "<p style='color: red;'>Error: Modelo no cargado correctamente</p>"

    except Exception as err:
        result_label.value = f"<p style='color: red;'>Error: {err}</p>"

    countdown_progress.value = 5

# Conectar directamente el botón sin threading
play_button.on_click(countdown_and_capture)
new_game_button.layout.display = ''  # Mostrar el botón



def on_play_button_clicked(b):
    with output:
        output.clear_output()
        countdown_and_capture()

def on_stats_button_clicked(b):
    total_games = player_wins + player_losses + player_ties
    win_percentage = (player_wins / total_games * 100) if total_games > 0 else 0

    stats_label.value = f"""
    <div style='background-color: #f0f0f0; padding: 10px; border-radius: 10px;'>
        <h3>Estadísticas del Juego</h3>
        <p>✅ Ganadas: {player_wins}</p>
        <p>❌ Perdidas: {player_losses}</p>
        <p>🤝 Empates: {player_ties}</p>
        <p>📈 Porcentaje de victorias: {win_percentage:.1f}%</p>
    </div>
    """

def on_exit_button_clicked(b):
    stats_label.value = "<p style='color: green;'>¡Gracias por jugar! Hasta pronto.</p>"
    # En un entorno real, aquí podrías guardar las estadísticas

# Conectar botones a funciones
play_button.on_click(on_play_button_clicked)
stats_button.on_click(on_stats_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Organizar la interfaz con paneles
button_box = widgets.HBox([play_button, stats_button, exit_button], layout=widgets.Layout(justify_content='center'))
game_panel = widgets.VBox([
    game_info_label,
    countdown_progress,
    button_box,
    image_display,
    result_label,
    stats_label
], layout=widgets.Layout(align_items='center'))

def on_new_game_clicked(b):
    result_label.value = ""
    game_info_label.value = "<h3>¡Bienvenido a Piedra, Papel o Tijera!</h3>"
    image_display.clear_output()
    countdown_progress.value = 5
    new_game_button.layout.display = 'none'

new_game_button.on_click(on_new_game_clicked)

game_panel = widgets.VBox([
    game_info_label,
    countdown_progress,
    button_box,
    image_display,
    result_label,
    stats_label,
    new_game_button   # nuevo botón
], layout=widgets.Layout(align_items='center'))



cod 13

In [ ]:
# La función countdown_and_capture se actualizó en el paso anterior
# para integrar la predicción del modelo y la lógica del juego.
# Este bloque de código confirma las conexiones y muestra la UI.

# Asegurarse de que el botón de jugar esté vinculado al proceso de cuenta regresiva y captura
# Esta conexión ya se realizó en el paso de creación de la UI de la subtarea anterior,
# pero lo reafirmamos aquí para mayor claridad en la subtarea de integración del flujo de juego.
play_button.on_click(on_play_button_clicked)

# El resto de los elementos de la UI y sus conexiones (estadísticas, salir) también provienen de
# la subtarea anterior y se asume que son funcionales para la UI general del juego.

# Mostrar la UI del juego para permitir la interacción


# Mostrar la interfaz
display(game_panel)
print("Interfaz cargada. Haz clic en '🎮 Jugar' para comenzar.")

# El flujo del juego ahora está integrado:
# 1. El usuario hace clic en 'Jugar'.
# 2. Se llama a on_play_button_clicked.
# 3. countdown_and_capture se inicia en un nuevo hilo.
# 4. countdown_and_capture realiza la cuenta regresiva y llama a take_photo.
# 5. take_photo captura la imagen.
# 6. La ruta de la imagen capturada se pasa a predict_player_choice.
# 7. predict_player_choice usa el modelo para obtener la elección predicha del jugador.
# 8. Se genera una elección aleatoria de la computadora.
# 9. Se llama a determine_winner con las elecciones del jugador y la computadora.
# 10. El resultado del juego se muestra en game_info_label.